<a href="https://colab.research.google.com/github/COMP-380-DATA-SCIENCE/leaf-disease-classifier/blob/main/LeafDiseaseIdentifierUIDraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.1 MB/s eta 

In [ ]:
#import necessary libraries. Gradio for the UI, transformers to load the model and torch to process the inputs and outputs
import gradio as gr
from transformers import ViTForImageClassification, ViTImageProcessor
import torch

In [ ]:
#Define the pretrained model
model_name = "Treelar/vit-b16-plant_village"

#Load the ViT model and the image processor
model = ViTForImageClassification.from_pretrained(model_name)
image_processor = ViTImageProcessor.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [ ]:
#The predict function takes an image input and returns the plant disease name as an output
def predict(image):

    #Convert the image into the models required format
    inputs = image_processor(images=image, return_tensors="pt")

    #Disable gradient calculation to make the process efficient
    with torch.no_grad():
      outputs = model(**inputs) #Gets the models output for the image
      logits = outputs.logits   #Output scores from the model

    #Converts the logits to a probability using ths softmax function
    probability = torch.nn.functional.softmax(logits, dim = 1)
    top_probability, top_index = probability.max(1) #Gets the highest probability with it's respective index

    #Gets the disease label from the model using the probability's index
    label = model.config.id2label[top_index.item()]
    result = f"{label}" #Result format
    return result

In [ ]:
#Gradio interface setup, it allows the users to upload the image and then it also gives feedback about the image
from gradio import Image, Label, Interface

def predict(image):
    # Your prediction code here
    prediction = "Picture of an orange huanglongbing leaf"  # Replace this with your actual prediction
    return image, prediction

interface = Interface(
    fn=predict,
    inputs=Image(label="Upload the Image"),
    outputs=["image", Label()],
    title="Plant Leaf Disease Identifier",
    description="Once the image has been uploaded and submitted, the disease of the plant will be determined."
)

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3e121319f57cd85a29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
interface.launch(debug=True) #Start server and launch the UI

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3e121319f57cd85a29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
